In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection


import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [17]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
# prod_type = "CE"
prod_type = "PE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


In [18]:
#########################
#User-defined functions #
#########################

def quad_func(x, a0, a1, a2):
    y = a0 + a1*x + a2*(np.square(x))
    return y

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       batch_size_divisor=10, no_of_epochs = 1000, stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    quad_polyn_volSurf = pd.read_csv(output_path + "A3_" + stock_ident + "_quadPolyn_vol_surface_params.csv")
    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week] 
            K = strike          
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]


            df_polyn_date = quad_polyn_volSurf[quad_polyn_volSurf['Date'] == sim_start_date.strftime("%d-%b-%Y")]

            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
    
            t_list = np.sort(np.unique(np.array(df_polyn_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
            
            df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
            polyn_coeff_wk = df_polyn_coeff[["a0", "a1", "a2"]].to_numpy().reshape(-1)

            x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)

            if (sim_moneyness <= x_min):
                vol_list = [quad_func(x_min, *polyn_coeff_wk)]
            elif (sim_moneyness >= x_max):
                vol_list = [quad_func(x_max, *polyn_coeff_wk)]
            else:
                vol_list = [quad_func(sim_moneyness, *polyn_coeff_wk)]

            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            put_strikes = np.sort(np.array(dict_pe_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
                        
            stock_vec = sim_stock_mat[:, 1]
            mny_list = stock_vec / K
            
            r_f = get_risk_free_rate_from_exact_date(sim_start_date)
            dt = float((each_month - sim_end_date).days) / 365
                    
            # Linear interpolation in varaince space across tenors
            if (week != no_of_weeks-1):
                
                # Time variables
                t1 = t_list[0]
                t2 = t_list[1] 
                t_star = dt
                
                # Vol at larger tenor: corresponding to maturity of option to be hedged
                
                df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[1]]
                polyn_coeff_mth = df_polyn_coeff[["a0", "a1", "a2"]].to_numpy().reshape(-1)
            
                x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)
                
                vol_t2 = quad_func(mny_list, *polyn_coeff_mth)
                vol_t2[mny_list <= x_min] = quad_func(x_min, *polyn_coeff_mth)
                vol_t2[mny_list >= x_max] = quad_func(x_max, *polyn_coeff_mth)
                #Variance corresponding to tenor t2
                var_t2 = np.square(vol_t2)
                
                #Vol at weekly tenor: corresponding to maturity of options used to hedge
                df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
                polyn_coeff_wk = df_polyn_coeff[["a0", "a1", "a2"]].to_numpy().reshape(-1)
                x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)
                
                vol_t1 = quad_func(mny_list, *polyn_coeff_wk)
                vol_t1[mny_list <= x_min] = quad_func(x_min, *polyn_coeff_wk)
                vol_t1[mny_list >= x_max] = quad_func(x_max, *polyn_coeff_wk)
                #Variance corresponding to tenor t1
                var_t1 = np.square(vol_t1)
                
                #Linear interpolation in variance space
                var_tstar = var_t1 + (var_t2 - var_t1) * ((t_star-t1)/(t2-t1))
                vol_arr = np.sqrt(var_tstar)
            else:
                vol_arr = np.array([0]) 
          
            opt_strike = strike

            if (option_type == "CE" and week != no_of_weeks-1):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt)
            elif (option_type == "PE" and week != no_of_weeks-1):
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt) 
            elif (option_type == "CE" and week == no_of_weeks-1):
                option_value = np.maximum(stock_vec - opt_strike, 0) 
            else:
                option_value = np.maximum(opt_strike - stock_vec, 0)
    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 
            
            
#             print("SKLearn Linear regression - OLS")
#             lin_model = LinearRegression().fit(x, y)
#             pfl_weights = lin_model.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model.intercept_
#             print("R-Square: ", lin_model.score(x,y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)           
            
#             print("SKLearn Linear regression - Ridge")
#             lin_model_ridge = Ridge(alpha=1.0).fit(x, y)
#             pfl_weights = lin_model_ridge.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model_ridge.intercept_
#             print("R-Square: ", lin_model_ridge.score(x, y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)
            
            
            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)
            
#          
#             #Find Optimal Alpha
#             alphas = np.array([-0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()
            
#             print("Stats Model _ OLS")
#             x = x
#             x = sm.add_constant(x)
#             lin_model2 = OLS(y,x).fit()
#             print(lin_model2.summary())
# #             print(lin_model2.params)
# #           pfl_weights = np.array(lin_model2.params).reshape(-1)
            
            
            ######################################################################################
            ### Portfolio weights of shorter term options from neural network of European options ###
            ######################################################################################
            
            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
            
            
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

        df_ce_pfl_weights.to_csv(output_path  + "B18A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
        df_pe_pfl_weights.to_csv(output_path + "B18A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
        df_cash_pfl_weights.to_csv(output_path  + "B18A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
    df_hedged_prod.to_csv(output_path + "B18A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [19]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

# #Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

# # #Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 

# # # The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)



2019-08-29  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.993579277167353
Coefficients:
[-0.         -0.01546168 -0.00643562 -0.00153193 -0.          0.
  0.          0.00755901  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.0005521   0.00687423  0.00436319  0.          0.
 -0.         -0.00682871 -0.00425198 -0.        ]
[12.5557858]
2019-08-29  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9965438818962865
Coefficients:
[-0.03018769 -0.         -0.          0.          0.00057713  0.00871538
  0.00429321  0.00310995  0.          0.          0.          0.
  0.          0.          0.00722546  0.02648993  0.00641697  0.00513219
  0.00597916  0.          0.          0.          0.          0.00049664]
[13.70580636]
2019-08-29  Week:  2
['6D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9950521727967292
Coefficients:
[-0.         -0.00706986 -0.01047114 -0.

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 33.70808731317095, tolerance: 13.032471269202032
  positive)


R-Square:  0.9974187341436058
Coefficients:
[-0.         -0.         -0.         -0.          0.          0.
  0.00123629  0.          0.         -0.01310475 -0.00314758 -0.
  0.          0.          0.          0.          0.          0.
  0.02924894  0.03462606  0.01696321  0.01187287  0.00462376  0.00460526
  0.00379258  0.00442238  0.00415829  0.00230162  0.          0.0013878
  0.00943981]
[13.6725645]
2019-10-31  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9994395206231868
Coefficients:
[-0.00522333 -0.00157387 -0.          0.          0.          0.0023528
  0.          0.         -0.         -0.01005356 -0.00343016 -0.
 -0.          0.          0.          0.          0.09191703  0.06455722
  0.04593395  0.0364261   0.0198899   0.00517464  0.01665005  0.01102182
  0.0026292   0.00785278  0.00628534  0.01069898]
[14.80722506]
2019-10-31  Week:  2
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9979706116596924
Coefficients:
[-0.         -0.

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79.57407885346106, tolerance: 34.8134313335891
  positive)


R-Square:  0.9833896204169047
Coefficients:
[-0.00934276 -0.          0.          0.          0.0041506   0.00071791
  0.0016171   0.0007738   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.00316163  0.          0.          0.
 -0.         -0.00523701 -0.          0.          0.00207847]
[3.01979126]
2019-10-31  Week:  4
['7D']
SKLearn Linear regression - Lasso
R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.]


...

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 852.9416098567744, tolerance: 623.9281899626977
  positive)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 60.8137218965407, tolerance: 32.42334538707802
  positive)


SKLearn Linear regression - Lasso
R-Square:  0.9987657108551575
Coefficients:
[-0.00234747 -0.00013366 -0.         -0.          0.          0.
  0.00036191  0.00046709  0.          0.         -0.         -0.00430366
 -0.00176066 -0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.00746116
  0.07327851  0.02351957  0.02662798  0.01765092  0.01318184  0.01162149
  0.00612417  0.00177006  0.00290726  0.00354778  0.00289224  0.001397
  0.0006133   0.0017207   0.00311878  0.00629387]
[8.23608011]
2020-06-25  Week:  2
['7D', '14D']
SKLearn Linear regression - Lasso
R-Square:  0.9958260479379377
Coefficients:
[-0.00347077 -0.         -0.          0.          0.          0.
  0.00083636  0.00086345  0.00066322  0.00050039  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.03551718  0.02682855  0.01213524  0.00812577  0.00

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74.59753529049397, tolerance: 73.2259739206477
  positive)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.1653776934702194, tolerance: 2.506393751879728
  positive)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.614206711548093, tolerance: 1.4551010807988696
  positive)


R-Square:  0.9935252809616867
Coefficients:
[ 0.          0.00041328  0.00150696  0.0001629   0.          0.
  0.         -0.         -0.01208596 -0.00154706 -0.          0.
  0.          0.          0.00076615  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.01400932  0.01204881  0.0007398   0.00375476
  0.00267817  0.00241035  0.00202191  0.00175436  0.00156994  0.00147969
  0.00129236  0.0010135   0.00090956  0.00103362  0.0005917 ]
[8.7459645]
2020-07-30  Week:  3
['7D', '14D']
SKLearn Linear regression - Lasso
R-Square:  0.9842119091990206
Coefficients:
[-2.45210669e-03 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.33893667e-05  1.42862981e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
